# Setting up Polaris Catalog

## Polaris Catalog configuration

Polaris requires you to configure principal users, roles and catalogs before you can begin creating tables.

For this workshop we'll use the following configuration:

1. Create a catalog named "polariscatalog" that we'll use to store our tables.
2. Create a principal user called "sparkuser". This user comes with clientId and clientSecret that we'll use to access the catalog on its behalf
3. The "sparkuser" will be assigned a principal role called "spark_principal_role" that will be associated with a catalog role
4. Create a catalog role called "catalog_role" and assign it permissions (like reading/writing table data)
5. Associate the catalog role to the principal role

The permission chain will look like this:

CATALOG_MANAGE_CONTENT --> catalog_role --> polariscatalog --> spark_principal_role --> sparkuser

In [96]:
import requests

# clientId and clientSecret are the catalog root credentials you can get from stdout when the catalog starts up
clientId="29f9ba8fa3b4cf58"
clientSecret="c4de93522fe0d1e229b51c057beb64d1"
accessToken=""

warehouseLocation="s3://upsolver-workshop-lake/sparkwarehouse/"
s3RoleArn="arn:aws:iam::765307950567:role/upsolver-workshop-s3-role"

catalogName="polariscatalog"
principalUser="sparkuser"
principalRoleName="spark_principal_role"
catalogRoleName="catalog_role"

In [ ]:
## GET ACCESS TOKEN

payload = {'grant_type':'client_credentials', 'client_id':clientId, 'client_secret':clientSecret, 'scope':'PRINCIPAL_ROLE:ALL'}

r = requests.post('http://polaris-catalog:8181/api/catalog/v1/oauth/tokens', data=payload)
accessToken=r.json()['access_token']
print(accessToken)

In [67]:
### CREATING THE CATALOG

payload = {
    'name':catalogName,
    'type':'INTERNAL',
    'properties':{ "default-base-location": warehouseLocation },
    'storageConfigInfo':{
        'roleArn':s3RoleArn,
        'storageType':'S3',
        'allowedLocations':[warehouseLocation]
    }
}

headers = {'Authorization':'Bearer '+ accessToken, 'Accept':'application/json'}

r = requests.post('http://polaris-catalog:8181/api/management/v1/catalogs', headers=headers, json=payload)
print(r.text)

In [ ]:
### CREATING THE PRINCIPAL

payload = {
    'name':principalUser,
    'type':'user'
}

headers = {'Authorization':'Bearer '+ accessToken, 'Accept':'application/json'}

r = requests.post('http://polaris-catalog:8181/api/management/v1/principals', headers=headers, json=payload)
print(r.text)

In [79]:
### CREATE PRINCIPAL ROLE

payload = {
    'principalRole':{'name':principalRoleName}
}

headers = {'Authorization':'Bearer '+ accessToken, 'Accept':'application/json'}

r = requests.post('http://polaris-catalog:8181/api/management/v1/principal-roles', headers=headers, json=payload)
print(r.text)

In [81]:
### ASSIGN PRINCIPAL ROLE TO PRINCIPAL

payload = {
    'principalRole':{'name':principalRoleName}
}

headers = {'Authorization':'Bearer '+ accessToken, 'Accept':'application/json'}

r = requests.put('http://polaris-catalog:8181/api/management/v1/principals/'+principalUser+'/principal-roles', headers=headers, json=payload)
print(r.text)

In [85]:
### CREATE A CATALOG ROLE FOR OUR CATALOG

payload = {
    'catalogRole':{'name':catalogRoleName}
}

headers = {'Authorization':'Bearer '+ accessToken, 'Accept':'application/json'}

r = requests.post('http://polaris-catalog:8181/api/management/v1/catalogs/'+catalogName+'/catalog-roles', headers=headers, json=payload)
print(r.text)

In [89]:
### ASSIGN THE CATALOG ROLE TO PRINCIPAL ROLE

payload = {
    'catalogRole':{'name':catalogRoleName}
}

headers = {'Authorization':'Bearer '+ accessToken, 'Accept':'application/json'}

r = requests.put('http://polaris-catalog:8181/api/management/v1/principal-roles/'+principalRoleName+'/catalog-roles/'+catalogName, headers=headers, json=payload)
print(r.text)

In [93]:
### GRANT PRIVILEGES TO CATALOG ROLE

payload = {
    'grant':{
        'type':'catalog',
        'privilege':'CATALOG_MANAGE_CONTENT'
    }
}

headers = {'Authorization':'Bearer '+ accessToken, 'Accept':'application/json'}

r = requests.put('http://polaris-catalog:8181/api/management/v1/catalogs/'+catalogName+'/catalog-roles/'+catalogRoleName+'/grants', headers=headers, json=payload)
print(r.text)